In [ ]:
!git commit -m"

### Add Dependencies

In [24]:
import sagemaker as sage
from sagemaker import get_execution_role

import zipfile
import os

from sagemaker import ModelPackage

!sudo apt install ffmpeg

sudo: apt: command not found


In [23]:
# Execution role
role = get_execution_role()
# S3 prefixes
common_prefix = "source_separation"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
# Sagemaker Session
sagemaker_session = sage.Session()
# Arn for Source Separator Model Package
modelpackage_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/source-separation-v11570291536-75ed8128ecee95e142ec4404d884ecad'



### Creating the Model

In [29]:
from sagemaker import ModelPackage
#from sagemaker.predictor import csv_serializer

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='application/x-recordio-protobuf')

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

### Running the Batch Job

In [27]:
TRANSFORM_WORKDIR = "input"

transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-075178354542/source_separation/batch-inference-input-data


In [30]:
import json 
import uuid

bucket = sagemaker_session.default_bucket()

transformer = model.transformer(1, 'ml.m4.xlarge', strategy='SingleRecord', output_path='s3://'+bucket+'/'+common_prefix+'/batch-transform-output')
transformer.transform(transform_input, content_type='application/x-recordio-protobuf')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

....................Starting the inference server with 4 workers.
[2020-04-12 06:13:02 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2020-04-12 06:13:02 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2020-04-12 06:13:02 +0000] [10] [INFO] Using worker: gevent
[2020-04-12 06:13:02 +0000] [14] [INFO] Booting worker with pid: 14
[2020-04-12 06:13:02 +0000] [15] [INFO] Booting worker with pid: 15
[2020-04-12 06:13:02 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-12 06:13:02 +0000] [17] [INFO] Booting worker with pid: 17
Testing...
Testing...
2020-04-12 06:13:46.428785: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
169.254.255.130 - - [12/Apr/2020:06:13:46 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:13:46 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586672026

169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"

Batch Transform output saved to s3://sagemaker-us-east-2-075178354542/source_separation/batch-transform-output


In [32]:
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'data/batch-transform-output/output-{}.zip'.format(i))

input1.mp3.out
input2.mp3.out
input3.mp3.out
input4.mp3.out
input5.mp3.out
input6.mp3.out
input7.mp3.out
input8.mp3.out
input9.mp3.out


In [33]:
for file in os.listdir('data/batch-transform-output'):
    print(file)
    with zipfile.ZipFile('data/batch-transform-output/'+file, 'r') as zip_ref:
        zip_ref.extractall('data/batch-transform-output/'+file.split('.')[0]+'/')

output-3.zip
output-6.zip
output-8.zip
output-9.zip
output-1.zip
output-4.zip
output-7.zip
output-2.zip
output-5.zip
